# Hidden Factor Hidden Topics
## Amazon Kindle Store Reviews Dataset
Dataset Link: http://jmcauley.ucsd.edu/data/amazon/
##### Sample Review

In [9]:

import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

import json

In [10]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
DATASET_SIZE = 17545
DATASET_FILE = 'train.json'
TEST_FILE = 'test.json'

In [11]:
dataset_file = open(DATASET_FILE)
dataset_lines = dataset_file.readlines()[:DATASET_SIZE]
data = []
for line in dataset_lines:
    data.append(json.loads(line)["reviewText"])
print data[0]

It suds up good, i used this in an attached sprayer bottle on my hose and it foamed up good spraying it out. Noticed after rinsing it immediately started beading water off which is a good sign. Only time will tell on how long it will last. As far as cleaning power it does fairly good, but i've seen better. Then again it is trying to do two jobs in one. The best cleaning of a car is in steps, but for an all in one cleaner and wax i think this outperforms most.


In [14]:
tf_vectorizer = CountVectorizer(strip_accents='unicode',
                                stop_words='english',
                                token_pattern=r'\b[a-zA-Z]{3,}\b',
                                max_df=0.5,
                                min_df=5
                               )
dtm_tf = tf_vectorizer.fit_transform(data)

In [15]:
print (tf_vectorizer.get_feature_names()[:10])
print (dtm_tf.shape)

[u'aaa', u'abilities', u'ability', u'able', u'abrasive', u'abs', u'absolute', u'absolutely', u'absorb', u'absorbant']
(17545, 7663)


In [16]:
NUMBER_OF_TOPICS = 5


lda_tf = LatentDirichletAllocation(n_components=NUMBER_OF_TOPICS, random_state=18)
lda_tf.fit(dtm_tf)

/Users/bmunot/anaconda2/lib/python2.7/site-packages/sklearn/decomposition/online_lda.py:536: DeprecationWarning: The default value for 'learning_method' will be changed from 'online' to 'batch' in the release 0.20. This warning was introduced in 0.18.
  DeprecationWarning)


LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7, learning_method=None,
             learning_offset=10.0, max_doc_update_iter=100, max_iter=10,
             mean_change_tol=0.001, n_components=5, n_jobs=1,
             n_topics=None, perp_tol=0.1, random_state=18,
             topic_word_prior=None, total_samples=1000000.0, verbose=0)

In [18]:
dataset_file = open(DATASET_FILE)
dataset_lines = dataset_file.readlines()[:DATASET_SIZE]

ReviewsByUser = {}
ItemReviews = {}
AverageRatingsByUsers = {}
AverageRatingsByItems = {}
UserHistory = {}
GlobalAverage = 0.0
for line in dataset_lines:
    d = json.loads(line)
    user = d["reviewerID"]
    item = d["asin"]
    try: 
        ReviewsByUser[user] += ' ' + d["reviewText"]
    except:
        ReviewsByUser[user] = d["reviewText"]
    try: 
        ItemReviews[item] += ' ' + d["reviewText"]
    except:
        ItemReviews[item] = d["reviewText"]
    try:
        AverageRatingsByUsers[user] = (AverageRatingsByUsers[user][0]+float(d["overall"]), AverageRatingsByUsers[user][1]+1)
    except:
        AverageRatingsByUsers[user] = (float(d["overall"]), 1)
    try:
        AverageRatingsByItems[item] = (AverageRatingsByItems[item][0]+float(d["overall"]), AverageRatingsByItems[item][1]+1)
    except:
        AverageRatingsByItems[item] = (float(d["overall"]), 1)
    try:
        UserHistory[user].append(item) 
    except:
        UserHistory[user] = [item]
    
    GlobalAverage += float(d["overall"])

GlobalAverage /= DATASET_SIZE   

for k in AverageRatingsByUsers.keys():
    AverageRatingsByUsers[k] = AverageRatingsByUsers[k][0]*1.0/AverageRatingsByUsers[k][1]        
        
for k in AverageRatingsByItems.keys():
    AverageRatingsByItems[k] = AverageRatingsByItems[k][0]*1.0/AverageRatingsByItems[k][1]

# print AverageRatingsByItems
# print AverageRatingsByUsers
print GlobalAverage
print "done"

4.4742661727
done


In [19]:
import numpy as np

WordToTopicMapping = {}

tf_feature_names = tf_vectorizer.get_feature_names()

rows = NUMBER_OF_TOPICS # topics count
columns = len(tf_feature_names) # Total number of words
    
M = np.zeros((rows, columns))
total_of_columns = np.sum(lda_tf.components_, axis=0)
print len(total_of_columns)
for i in range(columns):
    column_sum = total_of_columns[i]
    for j in range(rows):
        M[j][i] = lda_tf.components_[j][i]*1.0/column_sum

M = np.transpose(M)
DocVecRep = CountVectorizer(vocabulary=tf_feature_names) # document vector representation

7663


In [20]:
# print M

def MakeProbabilityDistribution(Gamma):
    column_sums = np.sum(Gamma, axis=0)
    return np.divide(Gamma, 1.0*column_sums)

Gamma_i = np.zeros((len(ItemReviews.keys()), NUMBER_OF_TOPICS)) #   #items   X  topics

j = 0
# Calculating latent factors for items
ItemToIndexMapping = {}
for item in ItemReviews.keys():
    ItemToIndexMapping[item] = j
    review = ItemReviews[item]
    review_vector = DocVecRep.fit_transform([review]).toarray()
    Gamma_i[j] = np.matmul(review_vector, M)
    j += 1


Gamma_i = np.transpose(Gamma_i)
Gamma_i = MakeProbabilityDistribution(Gamma_i)
print Gamma_i.shape

Gamma_u = np.zeros((len(ReviewsByUser.keys()), NUMBER_OF_TOPICS))
j = 0
# Calculating latent factors for users
UserToIndexMapping = {}
for user in ReviewsByUser.keys():
    UserToIndexMapping[user] = j
    review = ReviewsByUser[user]
    review_vector = DocVecRep.fit_transform([review]).toarray()
    Gamma_u[j] = np.matmul(review_vector, M)
    j += 1

Gamma_u = np.transpose(Gamma_u)
Gamma_u = MakeProbabilityDistribution(Gamma_u)
Gamma_u = np.transpose(Gamma_u)
print Gamma_u.shape

# print UserToIndexMapping
# print ItemToIndexMapping

    

(5, 1835)
(2928, 5)


In [21]:

total_users = len(ReviewsByUser.keys())
total_items = len(ItemReviews.keys())

ActualRatingAvailability = np.zeros((total_users, total_items))

dataset_file = open(DATASET_FILE)
dataset_lines = dataset_file.readlines()[:DATASET_SIZE]

for line in dataset_lines:
    d = json.loads(line)
    userId = d["reviewerID"]
    itemId = d["asin"]
    user = UserToIndexMapping[userId]
    item = ItemToIndexMapping[itemId]

In [23]:
def PredictUserToItemRating(userId, itemId):
    userIndex = UserToIndexMapping[userId]
    itemIndex = ItemToIndexMapping[itemId]
    gamma_user = Gamma_u[userIndex]
    gamma_item = Gamma_i[:, itemIndex]
    return np.dot(gamma_user, gamma_item)

def IsActuallyRated(userId, itemId):
    user = UserToIndexMapping[userId]
    item = ItemToIndexMapping[itemId]
    
    return ActualRatingAvailability[user][item] == 1

RECOMMENDATIONS = 5

def RecommendItemsToUser(userId):
    itemsList = ItemToIndexMapping.keys()
    predictions = []
    for item in itemsList:
        if not IsActuallyRated(userId, item):
            predictions.append((item, PredictUserToItemRating(userId, item)))
    sorted_by_predictions = sorted(predictions, key=lambda tup: tup[1], reverse=True)
    print len(sorted_by_predictions)
    print sorted_by_predictions[:min(RECOMMENDATIONS, len(predictions))]
    return sorted_by_predictions[:min(RECOMMENDATIONS, len(predictions))]

def GetRecommendationsForUser(userId):
    predictions = RecommendItemsToUser(userId)
    return [_[0] for _ in predictions]

def GetEvaluationOnTestData():
    test_file = open(TEST_FILE)
    dataset_lines = test_file.readlines()
    
    y_pred = []
    y_actual = []
    
    for line in dataset_lines:
        d = json.loads(line)
        userId = d["reviewerID"]
        itemId = d["asin"]
        #print PredictUserToItemRating(userId, itemId)
        prediction = AverageRatingsByUsers[userId] + AverageRatingsByItems[itemId] - GlobalAverage + PredictUserToItemRating(userId, itemId)
        y_pred.append(prediction)
        y_actual.append(d["overall"])
    MAE = round(mean_absolute_error(y_actual, y_pred), 2)
    RMSE = round(np.sqrt(mean_squared_error(y_actual, y_pred)), 2)
    return (MAE, RMSE)

USER_ID = 'A00473363TJ8YSZ3YAGG9'
print GetEvaluationOnTestData()
history = UserHistory[USER_ID]
recommendations = GetRecommendationsForUser(USER_ID)
print recommendations
print history

(0.69, 1.0)
1835
[(u'B007VJ2KEC', 0.2375639448734676), (u'B0051MYL8E', 0.23689234069258153), (u'B001LTVCBU', 0.23682540016636519), (u'B0043GCVUY', 0.23679141567699283), (u'B004VB5JDW', 0.23636302059635747)]
[u'B007VJ2KEC', u'B0051MYL8E', u'B001LTVCBU', u'B0043GCVUY', u'B004VB5JDW']
[u'B00353PWAW', u'B0062BHOOG', u'B0014FKI1Q', u'B000EH4V18']


In [25]:
pyLDAvis.sklearn.prepare(lda_tf, dtm_tf, tf_vectorizer)

/Users/bmunot/anaconda2/lib/python2.7/site-packages/pyLDAvis/_prepare.py:387: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  topic_term_dists = topic_term_dists.ix[topic_order]


PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
1      25.062409        1       1  0.045798  0.018435
2      22.538666        1       2 -0.001786 -0.221652
3      22.029966        1       3  0.042315  0.033419
4      16.995529        1       4  0.170053  0.095862
0      13.373431        1       5 -0.256381  0.073936, topic_info=     Category         Freq      Term        Total  loglift  logprob
term                                                               
510   Default  2823.000000   battery  2823.000000  30.0000  30.0000
4465  Default  2114.000000       oil  2114.000000  29.0000  29.0000
906   Default  6994.000000       car  6994.000000  28.0000  28.0000
2497  Default  1745.000000    filter  1745.000000  27.0000  27.0000
7428  Default  1749.000000       wax  1749.000000  26.0000  26.0000
7416  Default  2200.000000     water  2200.000000  25.0000  25.0000
2535  Default  1771.000000       fit  1771.000000  24.0000  24.0000
3709  Default  1104.000000   leather  1104.000000  23.0000  23.0000
7394  Default  1303.000000      wash  1303.000000  22.0000  22.0000
7532  Default  1153.000000     wiper  1153.000000  21.0000  21.0000
4922  Default  1221.000000     power  1221.000000  20.0000  20.0000
4612  Default  1194.000000     paint  1194.000000  19.0000  19.0000
3768  Default  2244.000000     light  2244.000000  18.0000  18.0000
6363  Default  1113.000000     spray  1113.000000  17.0000  17.0000
6917  Default  1239.000000      tire  1239.000000  16.0000  16.0000
7003  Default   987.000000   trailer   987.000000  15.0000  15.0000
603   Default   964.000000     blade   964.000000  14.0000  14.0000
3776  Default  1000.000000    lights  1000.000000  13.0000  13.0000
604   Default   949.000000    blades   949.000000  12.0000  12.0000
798   Default   791.000000     bulbs   791.000000  11.0000  11.0000
2109  Default  3175.000000      easy  3175.000000  10.0000  10.0000
1116  Default  1705.000000     clean  1705.000000   9.0000   9.0000
1121  Default   997.000000  cleaning   997.000000   8.0000   8.0000
1015  Default   848.000000    charge   848.000000   7.0000   7.0000
1017  Default   841.000000   charger   841.000000   6.0000   6.0000
5973  Default   904.000000     shine   904.000000   5.0000   5.0000
6979  Default   896.000000    towels   896.000000   4.0000   4.0000
3397  Default  1216.000000   install  1216.000000   3.0000   3.0000
7167  Default  1037.000000      unit  1037.000000   2.0000   2.0000
2047  Default   830.000000       dry   830.000000   1.0000   1.0000
...       ...          ...       ...          ...      ...      ...
5685   Topic5   195.047633      rust   255.183495   1.7432  -6.0859
2998   Topic5   370.781307    handle   637.470138   1.4700  -5.4435
6148   Topic5   655.747807     small  1505.435914   1.1808  -4.8734
4790   Topic5   423.819437     place   798.416278   1.3786  -5.3099
6373   Topic5   187.526805    spring   244.408112   1.7470  -6.1252
3152   Topic5   271.883669      home   436.307325   1.5389  -5.7538
1588   Topic5   167.213993   cutting   209.195055   1.7879  -6.2399
1945   Topic5   799.496514      does  2538.771509   0.8564  -4.6752
699    Topic5   361.915252       box   716.245940   1.3293  -5.4677
4100   Topic5   288.394231     metal   562.513089   1.3438  -5.6948
3486   Topic5   278.097826       isn   536.808499   1.3542  -5.7312
3182   Topic5   373.529204      hose   941.629467   1.0873  -5.4362
480    Topic5   247.037424       bar   461.530731   1.3869  -5.8496
6000   Topic5   221.328900     short   368.002419   1.5035  -5.9595
6082   Topic5   338.390752      size   949.161090   0.9805  -5.5350
3495   Topic5   296.726582      item   733.823435   1.1064  -5.6663
2186   Topic5   323.558725       end   970.566921   0.9134  -5.5798
4312   Topic5   459.957367      need  2410.388797   0.3555  -5.2280
7223   Topic5   685.784601       use  7066.189587  -0.3206  -4.8286
7375   Topic5   352.572096      want  1318.

In [27]:

# Converting json to votes file format
DATASET_FILE = 'test.json'
dataset_file = open(DATASET_FILE)
dataset_lines = dataset_file.readlines()

output_file = open('test.votes', 'w')
for line in dataset_lines:
    d = json.loads(line)
    output_file.write(d["reviewerID"] + ' ' + d["asin"]+ ' ' + str(d["overall"]) + ' ' + str(d["unixReviewTime"]) + ' ' + str(len(d["reviewText"].split())) + ' ' + d["reviewText"] + '\n')
output_file.close()

In [24]:
def GetURL(products_list):
    print products_list
    products = ','.join([str(_) for _ in products_list])
    url = "http://www.asinlab.com/php/convertfromasin.php?asin_num=" + products + "&id_type=UPC&bulk=true&x=false"
    return url
    
print GetURL(recommendations)
print GetURL(history)

[u'B007VJ2KEC', u'B0051MYL8E', u'B001LTVCBU', u'B0043GCVUY', u'B004VB5JDW']
http://www.asinlab.com/php/convertfromasin.php?asin_num=B007VJ2KEC,B0051MYL8E,B001LTVCBU,B0043GCVUY,B004VB5JDW&id_type=UPC&bulk=true&x=false
[u'B00353PWAW', u'B0062BHOOG', u'B0014FKI1Q', u'B000EH4V18']
http://www.asinlab.com/php/convertfromasin.php?asin_num=B00353PWAW,B0062BHOOG,B0014FKI1Q,B000EH4V18&id_type=UPC&bulk=true&x=false
